In [1]:
#!pip list


import sys
print(sys.version)



# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


count = 0
for root, folders, filenames in os.walk('/kaggle/input'):
   print(root, folders)

3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
/kaggle/input/lstmfile/tokenizer.pickle
/kaggle/input/lstmfile/baselineModel.h5
/kaggle/input/recursivebasefile1/tokenizer.pickle
/kaggle/input/recursivebasefile1/baselineModel.h5
/kaggle/input/newencoded/encodedNew.csv
/kaggle/input ['lstmfile', 'recursivebasefile1', 'newencoded']
/kaggle/input/lstmfile []
/kaggle/input/recursivebasefile1 []
/kaggle/input/newencoded []


In [2]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        batch_input_shape=[batch_size, None]
    ))
    model.add(tf.keras.layers.LSTM(
        units=rnn_units,
        return_sequences=True,
        stateful=True, #look into this causing the issue potentially, originally was True
        recurrent_initializer=tf.keras.initializers.GlorotNormal() #used to be be tf.keras.initializers.GlorotNormal() before downgradingf
    )) # temp removed
    model.add(tf.keras.layers.Dense(vocab_size))
    
    return model

In [3]:
#imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf # ML/DL
import keras
from keras.models import Sequential
from tensorflow.keras.layers import LSTM
from keras.layers import Activation, Dense, Dropout
import matplotlib.pyplot as plt
import tqdm
import pickle

#tf.compat.v1.enable_eager_execution()
import tensorflow_addons as tfa
#import coremltools as ct

In [4]:
#try: #for recursive model training
#    df = pd.read_csv(os.path.join('./WORKING'))
#    print('Wokrgin')
#except:
df = pd.read_csv(os.path.join('../input/newencoded/encodedNew.csv'))
#OG DATAFRAME
df.head()

,d/l,pu,tpu,rpu,su,tsu,rsu,sq,tsq,rsq,bp,tbp,rbp
0,4,8,10,18,25,35,42,56,70,2,63,80,0
1,1,3,25,28,3,15,61,7,50,6,6,50,25
2,2,21,55,19,24,70,10,10,30,2,13,40,27
3,2,12,30,45,12,35,6,8,25,7,28,80,0
4,4,11,15,53,12,15,5,20,25,36,48,60,17


In [5]:
import random

def alterRange(pu,tpu): #THE MODEL DOESNT CONTAIN ALL THE VOCAB WE ARE ASKING IT TO, EITHER MAP TO CLOSEST VOCAB OR RETAIN THE MOBEL ON MORE POSSIBLE NUMBER
    mul = pu/tpu
    alterMul = mul + 1.0 #should be 0.03 but ill make it 0.5 for now
    pu = alterMul * tpu  
    return int(float(pu))


def generateAction(diffLevel):
    if diffLevel == 5: 
        multiplier = random.uniform(0.9,1)
        tpa = random.randrange(10,80,5)
        numActions = int(tpa * multiplier)
        rpa = random.randrange(0,80-int(tpa))
    elif diffLevel == 4: 
        multiplier = random.uniform(0.7,0.8)
        tpa = random.randrange(10,80,5)
        numActions = int(tpa * multiplier)
        rpa = random.randrange(0,80-int(tpa))
        rpa = random.randrange(0,80-int(tpa))
    elif diffLevel == 3: 
        multiplier = random.uniform(0.5,0.6)
        tpa = random.randrange(10,80,5)
        numActions = int(tpa * multiplier)
        rpa = random.randrange(0,80-int(tpa))
        rpa = random.randrange(0,80-int(tpa))
    elif diffLevel == 2: 
        multiplier = random.uniform(0.3,0.4)
        tpa = random.randrange(10,80,5)
        numActions = int(tpa * multiplier)
        rpa = random.randrange(0,80-int(tpa))
        rpa = random.randrange(0,80-int(tpa))
    elif diffLevel == 1: 
        multiplier = random.uniform(0.1,0.2)
        tpa = random.randrange(10,80,5)
        numActions = int(tpa * multiplier)
        rpa = random.randrange(0,80-int(tpa))
    return tpa,numActions,rpa

def generateExamples(diffLevel, alter=False):
    if diffLevel == 5: 
        tpu,pu,rpu = generateAction(5)
        tsu,su,rsu = generateAction(5)
        tsq,sq,rsq = generateAction(5)
        tbp,bp,rbp = generateAction(5)
        print(pu)
        #temp
        if(alter == True):
            pu = alterRange(pu,tpu)
        #temp
        print(pu)
        outputStr = (str(diffLevel) + "-DL:~" + str(pu) + "~" + str(tpu) + "~" + str(rpu) + "~" + str(su) + "~" + str(tsu) + "~" + str(rsu) + "~" + str(sq) + "~" + str(tsq) + "~" + str(rsq) + "~"  +  str(bp) + "~" + str(tbp) + "~" + str(rbp) + "~" + "␣" )
        #outputStr = (str(diffLevel) + "-DL:~" + str(pu) + "~" + str(tpu) + "~" + str(rpu) + "~" + str(su) + "~" + str(tsu) + "~" + str(rsu) + "~" + str(sq) + "~" + str(tsq) + "~" + str(rsq) + "~" + "␣" )
    elif diffLevel == 4: 
        tpu,pu,rpu = generateAction(4)
        tsu,su,rsu = generateAction(4)
        tsq,sq,rsq = generateAction(4)
        tbp,bp,rbp = generateAction(4)
        outputStr = (str(diffLevel) + "-DL:~" + str(pu) + "~" + str(tpu) + "~" + str(rpu) + "~" + str(su) + "~" + str(tsu) + "~" + str(rsu) + "~" + str(sq) + "~" + str(tsq) + "~" + str(rsq) + "~"  +  str(bp) + "~" + str(tbp) + "~" + str(rbp) + "~" + "␣" )
    elif diffLevel == 3: 
        tpu,pu,rpu = generateAction(3)
        tsu,su,rsu = generateAction(3)
        tsq,sq,rsq = generateAction(3)
        tbp,bp,rbp = generateAction(3)
        outputStr = (str(diffLevel) + "-DL:~" + str(pu) + "~" + str(tpu) + "~" + str(rpu) + "~" + str(su) + "~" + str(tsu) + "~" + str(rsu) + "~" + str(sq) + "~" + str(tsq) + "~" + str(rsq) + "~"  +  str(bp) + "~" + str(tbp) + "~" + str(rbp) + "~" + "␣" )
    elif diffLevel == 2: 
        tpu,pu,rpu = generateAction(2)
        tsu,su,rsu = generateAction(2)
        tsq,sq,rsq = generateAction(2)
        tbp,bp,rbp = generateAction(2)
        outputStr = (str(diffLevel) + "-DL:~" + str(pu) + "~" + str(tpu) + "~" + str(rpu) + "~" + str(su) + "~" + str(tsu) + "~" + str(rsu) + "~" + str(sq) + "~" + str(tsq) + "~" + str(rsq) + "~"  +  str(bp) + "~" + str(tbp) + "~" + str(rbp) + "~" + "␣" )
    elif diffLevel == 1: 
        tpu,pu,rpu = generateAction(1)
        tsu,su,rsu = generateAction(1)
        tsq,sq,rsq = generateAction(1)
        tbp,bp,rbp = generateAction(1)
        outputStr = (str(diffLevel) + "-DL:~" + str(pu) + "~" + str(tpu) + "~" + str(rpu) + "~" + str(su) + "~" + str(tsu) + "~" + str(rsu) + "~" + str(sq) + "~" + str(tsq) + "~" + str(rsq) + "~"  +  str(bp) + "~" + str(tbp) + "~" + str(rbp) + "~" + "␣" )
    return outputStr
    
    

        
    

In [6]:

STOP_SIGN = "␣"   
allWorkoutsString = []
stringVocab = []

#import random


##NEW SHUFFLING TO FIX POOR EARLY PERFORMANCE
print(df.head())
df = df.sample(frac = 1)
print(df.head())
##NEW SHUFFLING TO FIX POOR EARLY PERFORMANCE

for i in range(len(df)):
    df.iloc[i]["pu"] = alterRange(df.iloc[i]["pu"],df.iloc[i]["tpu"]) #right now it overall makes pu way more diffiuclt
    workout = (str(df.iloc[i]["d/l"]) + "-DL:~" + str(df.iloc[i]["pu"]) + "~" + str(df.iloc[i]["tpu"]) + "~" + str(df.iloc[i]["rpu"]) + "~" + str(df.iloc[i]["su"]) + "~" + str(df.iloc[i]["tsu"]) + "~" + str(df.iloc[i]["rsu"]) + "~" + str(df.iloc[i]["sq"]) + "~" + str(df.iloc[i]["tsq"]) + "~" + str(df.iloc[i]["rsq"]) + "~"  +  str(df.iloc[i]["bp"]) + "~" + str(df.iloc[i]["tbp"]) + "~" + str(df.iloc[i]["rbp"]) + "~" + "␣" )
    allWorkoutsString.append(workout)
    #stringVocab.append(workout)
      
        

#df.to_csv("./WORKING")
#
#for z in range(50):   
    #allWorkoutsString.append(generateExamples(5,True))
    #allWorkoutsString.append(generateExamples(4))
    #allWorkoutsString.append(generateExamples(3))
    #allWorkoutsString.append(generateExamples(2))
    #allWorkoutsString.append(generateExamples(1))
    
#allWorkoutsString.append(generateExamples(5,True))



   d/l  pu  tpu  rpu  su  tsu  rsu  sq  tsq  rsq  bp  tbp  rbp
0    4   8   10   18  25   35   42  56   70    2  63   80    0
1    1   3   25   28   3   15   61   7   50    6   6   50   25
2    2  21   55   19  24   70   10  10   30    2  13   40   27
3    2  12   30   45  12   35    6   8   25    7  28   80    0
4    4  11   15   53  12   15    5  20   25   36  48   60   17
      d/l  pu  tpu  rpu  su  tsu  rsu  sq  tsq  rsq  bp  tbp  rbp
984     1   4   20   15  12   75    2   4   25   37   2   10   68
1011    4  19   25    9  35   45    9  39   50    4  63   80    0
1151    1   6   45    0   6   40    3  12   75    0   4   25    9
110     3  19   35   21  46   80    0  28   50   24  35   65   11
146     5  38   40   12  69   75    3  74   75    5  45   50   27


In [7]:
def decodeWorkout(input):    
    c = input.split("~")
    if(len(c) == 5):
        pushupStr = str(c[0] + "\n" + c[1] + " pushups for " + c[2] + " seconds with " + c[3] + " second rest ")
        outputStr = (pushupStr)
    elif(len(c) == 8):
        pushupStr = str(c[0] + "\n" + c[1] + " pushups for " + c[2] + " seconds with " + c[3] + " second rest ")
        situpStr = str("\n" + c[4] + " situps for " + c[5] + " seconds with " + c[6] + " second rest ")
        outputStr = (pushupStr + situpStr)
    elif(len(c)==11):
        pushupStr = str(c[0] + "\n" + c[1] + " pushups for " + c[2] + " seconds with " + c[3] + " second rest ")
        situpStr = str("\n" + c[4] + " situps for " + c[5] + " seconds with " + c[6] + " second rest ")
        squatStr = str("\n" + c[7] + " squats for " + c[8] + " seconds with " + c[9] + " second rest ")
        outputStr = (pushupStr + situpStr + squatStr)
    elif(len(c) == 14):
        pushupStr = str(c[0] + "\n" + c[1] + " pushups for " + c[2] + " seconds with " + c[3] + " second rest ")
        situpStr = str("\n" + c[4] + " situps for " + c[5] + " seconds with " + c[6] + " second rest ")
        squatStr = str("\n" + c[7] + " squats for " + c[8] + " seconds with " + c[9] + " second rest ")
        burpeeStr = str("\n" + c[10] + " burpees for " + c[11] + " seconds with " + c[12] + " second rest ")
        outputStr = (pushupStr + situpStr + squatStr + burpeeStr)
    else:
        outputStr = "fail"
    
    return outputStr
#for i in range(15) :
#        print(decodeWorkout(allWorkoutsString[i]))


In [8]:
#encoding each of the characters into integers so the model can understand -> paying special attention to the stop_sign
import pickle

with open('../input/lstmfile/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)


print(tokenizer.get_config())
VOCABULARY_SIZE = len(tokenizer.word_counts) + 1 

dataset_vectorized = tokenizer.texts_to_sequences(allWorkoutsString)
print('Vectorized dataset size', len(dataset_vectorized))


for i in range(10):
    print(tokenizer.texts_to_sequences([f"{i}"]))



{'num_words': None, 'filters': '', 'lower': False, 'split': '', 'char_level': True, 'oov_token': None, 'document_count': 1500, 'word_counts': '{"5": 5707, "-": 1500, "D": 1500, "L": 1500, ":": 1500, "~": 19500, "2": 4330, "7": 1901, "3": 3542, "0": 4869, "1": 4829, "6": 2221, "4": 3507, "\\u2423": 1500, "9": 1132, "8": 1554}', 'word_docs': '{"5": 1472, "~": 1500, "L": 1500, "0": 1479, "1": 1435, "6": 1182, "-": 1500, "3": 1347, ":": 1500, "4": 1360, "2": 1406, "7": 1095, "\\u2423": 1500, "D": 1500, "9": 835, "8": 1005}', 'index_docs': '{"2": 1472, "1": 1500, "13": 1500, "3": 1479, "4": 1435, "8": 1182, "11": 1500, "6": 1347, "14": 1500, "7": 1360, "5": 1406, "9": 1095, "15": 1500, "12": 1500, "16": 835, "10": 1005}', 'index_word': '{"1": "~", "2": "5", "3": "0", "4": "1", "5": "2", "6": "3", "7": "4", "8": "6", "9": "7", "10": "8", "11": "-", "12": "D", "13": "L", "14": ":", "15": "\\u2423", "16": "9"}', 'word_index': '{"~": 1, "5": 2, "0": 3, "1": 4, "2": 5, "3": 6, "4": 7, "6": 8, "7

In [9]:
#add padding to ensure same length --- only if all are not the sane length
print(len(dataset_vectorized))
maxLength = 0
for workout_index, workout in enumerate(dataset_vectorized[:len(dataset_vectorized)]):
    #if(len(workout)) != 167:
    if(maxLength < len(workout)):
        #print('Workout #{} length: {}'.format(workout_index + 1, len(workout)))
        maxLength = len(workout)
#padding is required
print(maxLength)

1500
44


In [10]:
def decodeVector(vector):
    decodedWorkout = tokenizer.sequences_to_texts([vector])[0]
    print(decodedWorkout)
    
#basic test to see if the decoder is working
decodeVector(dataset_vectorized[0])

1 - D L : ~ 2 4 ~ 2 0 ~ 1 5 ~ 1 2 ~ 7 5 ~ 2 ~ 4 ~ 2 5 ~ 3 7 ~ 2 ~ 1 0 ~ 6 8 ~ ␣


In [11]:
dataset_vectorized_padded_without_stops = tf.keras.preprocessing.sequence.pad_sequences(
    dataset_vectorized,
    padding='post',
    truncating='post',
    # We use -1 here and +1 in the next step to make sure
    # that all recipes will have at least 1 stops sign at the end,
    # since each sequence will be shifted and truncated afterwards
    # (to generate X and Y sequences).
    maxlen=maxLength-1,
    #value=tokenizer.texts_to_sequences([STOP_SIGN])[0] #maybe append here again
)
dataset_vectorized_padded = tf.keras.preprocessing.sequence.pad_sequences(
    dataset_vectorized_padded_without_stops,
    padding='post',
    truncating='post',
    maxlen=maxLength+1,
    #value=tokenizer.texts_to_sequences([STOP_SIGN])[0]
)

for workout_index, workout in enumerate(dataset_vectorized_padded[:len(dataset_vectorized_padded)]):
    if(len(workout)) != maxLength+1:
        print('Workout #{} length: {}'.format(workout_index + 1, len(workout)))

#test it out
decodeVector(dataset_vectorized_padded[0])

1 - D L : ~ 2 4 ~ 2 0 ~ 1 5 ~ 1 2 ~ 7 5 ~ 2 ~ 4 ~ 2 5 ~ 3 7 ~ 2 ~ 1 0 ~ 6 8 ~ ␣


In [12]:
dataset = tf.data.Dataset.from_tensor_slices(dataset_vectorized_padded)

2022-04-24 20:59:28.700452: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 20:59:28.780325: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 20:59:28.781100: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 20:59:28.783179: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [13]:
#split processing
def split_input_target(workout):
    input_text = workout[:-1]
    target_text = workout[1:]
    
    return input_text, target_text
dataset_targeted = dataset.map(split_input_target)
print(dataset_targeted)

<MapDataset shapes: ((44,), (44,)), types: (tf.int32, tf.int32)>


In [14]:
def loss(labels, logits):
    entropy = tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )
    
    return entropy
#compile

adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #used to be 0.001 which worked much better


early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=5,
    monitor='loss',
    restore_best_weights=True,
    verbose=1
)

In [15]:
# Create a checkpoints directory. TO SAVE AND CONTINUE TRAINING
checkpoint_dir = 'tmp/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_') #used to be ckpt_{epoch}, got rid due to space issues on kaggle
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

tqdm_callback = tfa.callbacks.TQDMProgressBar()

In [16]:

def loss(labels, logits):
    entropy = tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )
    
    return entropy

#OG MODEL
#model = keras.models.load_model(
#    "../input/lstmfile/baselineModel.h5", custom_objects={"loss": loss}
#)


#New recursive approach
model = keras.models.load_model(
    "../input/recursivebasefile1/baselineModel.h5", custom_objects={"loss": loss}
)


#model = keras.models.load_model("../input/lstmfile/baselineModel.h5")


model.compile(
    optimizer=adam_optimizer,
    loss=loss
)



In [17]:
# Batch size.
BATCH_SIZE = 64
# Buffer size to shuffle the dataset (TF data is designed to work
# with possibly infinite sequences, so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in
# which it shuffles elements).
SHUFFLE_BUFFER_SIZE = 1000
dataset_train = dataset_targeted \
  .shuffle(SHUFFLE_BUFFER_SIZE) \
  .batch(BATCH_SIZE, drop_remainder=True) \
  .repeat()
print(dataset_train)

<RepeatDataset shapes: ((64, 44), (64, 44)), types: (tf.int32, tf.int32)>


In [18]:
#const
EPOCHS = 3 #5 seemed to work well
INITIAL_EPOCH = 1
STEPS_PER_EPOCH = len(allWorkoutsString) / BATCH_SIZE
print('EPOCHS:          ', EPOCHS) 
print('INITIAL_EPOCH:   ', INITIAL_EPOCH)
print('STEPS_PER_EPOCH: ', STEPS_PER_EPOCH) #default was 1500

EPOCHS:           3
INITIAL_EPOCH:    1
STEPS_PER_EPOCH:  23.4375


In [19]:
history = model.fit(
    x=dataset_train,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    initial_epoch=INITIAL_EPOCH,
    callbacks=[
        checkpoint_callback,
        #tqdm_callback
       # early_stopping_callback #we want to overfit this so get rid of early stopping
    ]
)

model.save("new/baselineModel.h5")

Epoch 2/3


2022-04-24 20:59:33.887293: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-24 20:59:34.791004: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


23/23 [==============================] - 4s 27ms/step - loss: 0.9349
Epoch 3/3
23/23 [==============================] - 1s 28ms/step - loss: 0.3591


In [20]:
def generate_text(model, start_string, num_generate = 1000, temperature=1.0):
    # Evaluation step (generating text using the learned model)

    padded_start_string = start_string
    # Converting our start string to numbers (vectorizing).
    input_indices = np.array(tokenizer.texts_to_sequences([padded_start_string]))
    # Empty string to store our results.
    text_generated = []
    # Here batch size == 1.
   # model.reset_states()
    for char_index in range(num_generate):
        predictions = model(input_indices)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        # Using a categorical distribution to predict the character returned by the model.
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(
            predictions,
            num_samples=1
        )[-1, 0].numpy()
        # We pass the predicted character as the next input to the model
        # along with the previous hidden state.
        input_indices = tf.expand_dims([predicted_id], 0)

        next_character = tokenizer.sequences_to_texts(input_indices.numpy())[0]
        text_generated.append(next_character)

       # output = ''.join(text_generated)

       # output = decodeWorkout(output)

    return (padded_start_string + ''.join(text_generated))

In [21]:
vocab_size=VOCABULARY_SIZE
embedding_dim=256
rnn_units=1024

simplified_batch_size = 1
model_simplified = build_model(vocab_size, embedding_dim, rnn_units, simplified_batch_size)
model_simplified.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model_simplified.build(tf.TensorShape([simplified_batch_size, None]))
model_simplified.summary()
#nonTrainedModel = keras.models.load_model("../input/lstmfile/simplified_model.h5")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 256)            4352      
_________________________________________________________________
lstm (LSTM)                  (1, None, 1024)           5246976   
_________________________________________________________________
dense (Dense)                (1, None, 17)             17425     
Total params: 5,268,753
Trainable params: 5,268,753
Non-trainable params: 0
_________________________________________________________________


In [22]:
output = (generate_text(model_simplified, '5-DL:', num_generate = maxLength, temperature=1.0)) #make sure you pass it model_simplified
print(output)
print(decodeWorkout(output))

#allWorkoutsString.append("5-DL:~60~50~28~14~15~17~15~15~36~34~35~5~␣")
#allWorkoutsString.append("5-DL:~40~30~6~10~10~5~44~45~6~41~45~16~␣")
#allWorkoutsString.append("5-DL:~75~65~9~24~25~45~41~45~34~72~80~0~␣")
#allWorkoutsString.append("5-DL:~85~70~0~55~55~3~33~35~23~30~30~34~␣")
#allWorkoutsString.append("5-DL:~60~50~22~68~75~4~63~70~6~14~15~33~␣")

5-DL:~57~30~29~14~15~27~56~60~17~35~35~23~␣
5-DL:
57 pushups for 30 seconds with 29 second rest 
14 situps for 15 seconds with 27 second rest 
56 squats for 60 seconds with 17 second rest 
35 burpees for 35 seconds with 23 second rest 


In [23]:
output = (generate_text(model_simplified, '4-DL:', num_generate = 43, temperature=1.0)) #make sure you pass it model_simplified
print(output)
print(decodeWorkout(output))

4-DL:~47~50~22~20~25~25~14~20~25~27~35~19~␣
4-DL:
47 pushups for 50 seconds with 22 second rest 
20 situps for 25 seconds with 25 second rest 
14 squats for 20 seconds with 25 second rest 
27 burpees for 35 seconds with 19 second rest 


In [24]:
output = (generate_text(model_simplified, '3-DL:', num_generate = maxLength, temperature=1.0)) #make sure you pass it model_simplified
print(output)
print(decodeWorkout(output))


3-DL:~39~50~20~27~50~22~18~30~9~42~80~0~␣
3-DL:
39 pushups for 50 seconds with 20 second rest 
27 situps for 50 seconds with 22 second rest 
18 squats for 30 seconds with 9 second rest 
42 burpees for 80 seconds with 0 second rest 


In [25]:
output = (generate_text(model_simplified, '2-DL:', num_generate = maxLength, temperature=1.0)) #make sure you pass it model_simplified
print(output)
print(decodeWorkout(output))

2-DL:~86~45~21~7~20~14~4~10~47~10~30~12~␣
2-DL:
86 pushups for 45 seconds with 21 second rest 
7 situps for 20 seconds with 14 second rest 
4 squats for 10 seconds with 47 second rest 
10 burpees for 30 seconds with 12 second rest 


In [26]:
output = (generate_text(model_simplified, '1-DL:', num_generate = maxLength, temperature=1.0)) #make sure you pass it model_simplified
print(output)
print(decodeWorkout(output))

1-DL:~36~30~23~12~70~6~9~45~14~11~55~16~␣
1-DL:
36 pushups for 30 seconds with 23 second rest 
12 situps for 70 seconds with 6 second rest 
9 squats for 45 seconds with 14 second rest 
11 burpees for 55 seconds with 16 second rest 


In [27]:
import shutil

shutil.rmtree('/tmp')

model_simplified.save("simplified_modelMOD.h5")